# Prepare

**import**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import sys
sys.path.append("./util_")
import acquire
import prepare

### get data

In [2]:
# get data from acquire file
dogs = acquire.get_data()
dogs.head()

,UniqueID,DateOfBite,Species,Breed,Age,Gender,SpayNeuter,Borough,ZipCode
0,1,January 01 2018,DOG,UNKNOWN,NaN,U,False,Brooklyn,11220
1,2,January 04 2018,DOG,UNKNOWN,NaN,U,False,Brooklyn,NaN
2,3,January 06 2018,DOG,Pit Bull,NaN,U,False,Brooklyn,11224
3,4,January 08 2018,DOG,Mixed/Other,4,M,False,Brooklyn,11231
4,5,January 09 2018,DOG,Pit Bull,NaN,U,False,Brooklyn,11224


## Check column data types

In [3]:
dogs.select_dtypes("object")

,DateOfBite,Species,Breed,Age,Gender,Borough,ZipCode
0,January 01 2018,DOG,UNKNOWN,NaN,U,Brooklyn,11220
1,January 04 2018,DOG,UNKNOWN,NaN,U,Brooklyn,NaN
2,January 06 2018,DOG,Pit Bull,NaN,U,Brooklyn,11224
3,January 08 2018,DOG,Mixed/Other,4,M,Brooklyn,11231
4,January 09 2018,DOG,Pit Bull,NaN,U,Brooklyn,11224
...,...,...,...,...,...,...,...
22658,December 24 2017,DOG,CHIWEENIE MIX,7,M,Staten Island,10303
22659,December 24 2017,DOG,DUNKER,5,F,Staten Island,10303
22660,December 21 2017,DOG,"Schnauzer, Miniature",10M,M,Staten Island,10312
22661,December 28 2017,DOG,Mixed/Other,NaN,F,Staten Island,10308


In [4]:
# convert string to datetime
dogs["DateOfBite"] = pd.to_datetime(dogs['DateOfBite'])

In [5]:
# # replace all the NaN cells with the string median
# dogs.Age = dogs['Age'].fillna("median")

In [6]:
# Define regex patterns
year_pattern = r'(\d+) *[yY](?:ear)?'
month_pattern = r'(\d+) *m(?:onth)?'
week_pattern = r'(\d+) *w(?:eek)?'

# Clean and extract values
# df['Age'] = dogs['Age'].astype(str)  # Convert column to string
dogs['year'] = dogs['Age'].str.extract(year_pattern, expand=False).astype(float).fillna(0)
dogs['month'] = dogs['Age'].str.extract(month_pattern, expand=False).astype(float).fillna(0) / 12
dogs['week'] = dogs['Age'].str.extract(week_pattern, expand=False).astype(float).fillna(0) / 52

# Adjust values for numeric years
dogs['year'] += dogs['Age'].str.replace(year_pattern, '', regex=True).astype(float).fillna(0)

# Replace empty string cells and cells with only whitespace with 0
dogs['Age'] = dogs['Age'].str.strip()  # Remove leading and trailing spaces
dogs['Age'].replace(r'^\s*$', '0', regex=True, inplace=True)  # Replace empty string cells with 0

# Drop original column
# dogs = dogs.drop('column', axis=1)

dogs

ValueError: could not convert string to float: ''

In [8]:
dogs.year.value_counts()

0.0     22649
4.0         3
10.0        2
3.0         2
2.0         2
13.0        1
68.0        1
5.0         1
17.0        1
6.0         1
Name: year, dtype: int64

In [42]:
# year labeled cells
yr = dogs.Age.str.contains(r"[yY]", regex=True)
dogs.Age.ste.replace()

0        False
1        False
2        False
3        False
4        False
         ...  
22658    False
22659    False
22660    False
22661    False
22662    False
Name: Age, Length: 22663, dtype: bool

In [22]:
# get all the cells that are not nulls
non_null_dogs = dogs[dogs.Age.notna()]

# find all year strings ages
y = non_null_dogs[non_null_dogs.Age.str.contains(r"[yY]", regex=True)]

# use extract to only get the numbers in the string cells representing yesr
y["good_year"] = y.Age.str.extract("(\d+)")
y.head()

/var/folders/jq/1gsqwxz13gxf_lpnwb_27r0c0000gn/T/ipykernel_2752/1284149062.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["good_year"] = y.Age.str.extract("(\d+)")


,UniqueID,DateOfBite,Species,Breed,Age,Gender,SpayNeuter,Borough,ZipCode,good_year
5,6,January 03 2018,DOG,BASENJI,4Y,M,False,Brooklyn,11231,4
8,9,January 04 2018,DOG,American Pit Bull Mix / Pit Bull Mix,5Y,M,False,Brooklyn,11235,5
9,10,January 10 2018,DOG,MIXED,3Y,F,False,Brooklyn,11208,3
26,27,January 08 2018,DOG,POODLE X,13Y,M,True,Brooklyn,11215,13
325,326,June 21 2018,DOG,Cane Corso,7 YRS,M,False,Brooklyn,11223,7


In [7]:
# find all month strings ages
m = non_null_dogs[non_null_dogs.Age.str.contains(r"[mM]", regex=True)]
m["good_year"] = m.Age.str.extract("(\d+)").astype(int) / 12
m.head()

/var/folders/jq/1gsqwxz13gxf_lpnwb_27r0c0000gn/T/ipykernel_2752/2989944671.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m["good_year"] = m.Age.str.extract("(\d+)").astype(int) / 12


,UniqueID,DateOfBite,Species,Breed,Age,Gender,SpayNeuter,Borough,ZipCode,good_year
46,47,February 01 2018,DOG,"Dachshund, Wirehaired, Miniature",10M,M,False,Brooklyn,NaN,0.833333
57,58,February 09 2018,DOG,Chihuahua,9M,M,False,Brooklyn,11233,0.750000
67,68,February 18 2018,DOG,POMERANIAN MIX,8M,M,False,Brooklyn,NaN,0.666667
106,107,March 11 2018,DOG,Pit Bull,4M,F,False,Brooklyn,11231,0.333333
120,121,March 21 2018,DOG,Maltese,7-8M,F,False,Brooklyn,11216,0.583333


In [8]:
# find all weeks strings ages
w = non_null_dogs[non_null_dogs.Age.str.contains(r"[KkwW]", regex=True)]
w["good_year"] = w.Age.str.extract("(\d+)").astype(int) / 54
w.head()


/var/folders/jq/1gsqwxz13gxf_lpnwb_27r0c0000gn/T/ipykernel_2752/2528106143.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  w["good_year"] = w.Age.str.extract("(\d+)").astype(int) / 54


,UniqueID,DateOfBite,Species,Breed,Age,Gender,SpayNeuter,Borough,ZipCode,good_year
107,108,March 09 2018,DOG,Labrador Retriever,10W,M,False,Brooklyn,11222,0.185185
1894,1895,September 16 2020,DOG,German Shepherd,11WKS,F,False,Brooklyn,11203,0.203704
4076,4077,May 20 2020,DOG,Shih Tzu,12WKSKS,F,True,Bronx,10453,0.222222
4411,4412,April 13 2021,DOG,Cocker Spaniel,16W,F,False,Bronx,10453,0.296296
5239,5240,July 24 2018,DOG,NaN,8W,U,False,Manhattan,10022,0.148148


In [9]:
non_null_dogs.Age.str.isnumeric()

3         True
5        False
8        False
9        False
11        True
         ...  
22654     True
22657     True
22658     True
22659     True
22660    False
Name: Age, Length: 11442, dtype: bool

In [10]:
len(non_null_dogs.Age)

11442

## Remove columns

remove ZipCode beacause most of my data has NaN cells.

In [11]:
# drop redundent columns
remove_cols = ["UniqueID", "ZipCode"]
dogs = prepare.drop_cols(dogs, remove_cols)

Original dataframe size: (22663, 9)
New dataframe size: (22663, 7)
